<a href="https://colab.research.google.com/github/leiyi-ihb/ColabPractics/blob/main/B_practices/keras_example06_CNN%E5%BA%94%E7%94%A8%E4%BA%8E%E6%89%8B%E5%86%99%E6%95%B0%E5%AD%97%E8%AF%86%E5%88%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# CNN应用于手写数字识别
# 尽管我没看出来这是手写数字识别，但我还是按作者的来取标题
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Convolution2D, MaxPool2D, Flatten
from keras.optimizers import Adam

In [3]:
# 载入数据
(x_train, y_train), (x_test, y_test)=mnist.load_data()
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

11490434/11490434 [==============================] - 0s 0us/step
(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [5]:
# 将(60000, 28, 28)转为4维的(60000, 28, 28, 1)。第一个维度为样本个数，第二、三维度分别是图片的长和宽，第四个维度是图片的深度，也就是图片的颜色（一般认为黑白图片深度为1，彩色图片深度为3）.这里用的是黑白图片，因此是1
x_train = x_train.reshape(-1, 28, 28, 1)/255 #-1表示会自动匹配，直至输入60000也行，除以255是为了归一化，将数据归一化到(0-1)【跟前面课时讲的一样】。
x_test = x_test.reshape(-1, 28, 28, 1)/255

# 转换label为one hot形式
from keras.utils import np_utils
print(y_train[0])
y_train = np_utils.to_categorical(y_train, num_classes=10) #这里为啥分为10个类没有讲清楚
y_test = np_utils.to_categorical(y_test, num_classes=10) #这里为啥分为10个类没有讲清楚
print(y_train[0])

5
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [6]:
from keras.layers.pooling.max_pooling2d import MaxPooling2D
from keras.layers.convolutional.conv1d import Convolution1D
model = Sequential()
model.add(Convolution2D(input_shape=(28, 28, 1), #设置的输入平面。可以发现，其实跟前面几节课相比，输入是很有规律的
                        filters=32, #卷积核，也叫滤波器，的个数
                        kernel_size=5, #卷积核窗口大小
                        strides=1, 
                        padding="same", 
                        activation="relu"))

#加池化层
model.add(MaxPooling2D(pool_size=2, 
                        strides=2, 
                        padding="same"
                        ))


#加第二个卷积层
model.add(Convolution2D(64, 5, #直接写数字，就是python的传参语法，第一个参数为filters, 第二个参数为kernel_size，当然也可以直接写出来
                        strides=1,
                        padding="same",
                        activation="relu"))

#加第二个池化层
model.add(MaxPooling2D(2, 2, "same"))

#将第二个池化层的输出扁平化为1维数据
model.add(Flatten())

#加第一个全连接层
model.add(Dense(1024, activation="relu"))

#加Dropout
model.add(Dropout(0.5))

#加第二个全连接层
model.add(Dense(10, activation="softmax"))

#定义优化器
adam = Adam(lr=1e-4)

#模型编译
model.compile(optimizer=adam, loss="categorical_crossentropy", metrics= ['accuracy'])

# 开始训练模型
model.fit(x_train, y_train, epochs=10, batch_size=64)

# 评估模型
loss, accuracy = model.evaluate(x_test, y_test)
print("\ntest loss:", loss)
print('accuracy:', accuracy)

loss, accuracy = model.evaluate(x_train, y_train) #将我们的训练集也评估一下
print("\ntrain loss:", loss)
print('accuracy:', accuracy)

Epoch 1/10
938/938 [==============================] - 124s 131ms/step - loss: 0.3430 - accuracy: 0.9024
Epoch 2/10
938/938 [==============================] - 121s 129ms/step - loss: 0.0925 - accuracy: 0.9724
Epoch 3/10
938/938 [==============================] - 122s 130ms/step - loss: 0.0615 - accuracy: 0.9812
Epoch 4/10
938/938 [==============================] - 120s 128ms/step - loss: 0.0511 - accuracy: 0.9842
Epoch 5/10
938/938 [==============================] - 122s 130ms/step - loss: 0.0415 - accuracy: 0.9872
Epoch 6/10
938/938 [==============================] - 123s 131ms/step - loss: 0.0348 - accuracy: 0.9887
Epoch 7/10
938/938 [==============================] - 133s 142ms/step - loss: 0.0307 - accuracy: 0.9908
Epoch 8/10
938/938 [==============================] - 126s 134ms/step - loss: 0.0270 - accuracy: 0.9916
Epoch 9/10
938/938 [==============================] - 122s 130ms/step - loss: 0.0237 - accuracy: 0.9924
Epoch 10/10
313/313 [==============================] - 6s 20ms/s

**结果：**使用卷积的时候，没有GPU，迭代速率会慢很多，都上百秒了，前几节课运行也就几十秒。不过按讲述者来说，结果就是好，卷积的学习结果就是好。